In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [97]:
import modulos as md
import numpy as np
from backtesting import Backtest
import pandas_ta as ta
from backtesting import Strategy
import pandas as pd
import sys
import os

def backtesting(data, plot_flag=False):
    balance = 100    
    def indicador(df_campo):
        indi=pd.Series(df_campo)
        return indi.to_numpy()
    def ema(df_campo):
        indi=ta.ema(df_campo.s,length=200)
        return indi.to_numpy()   
    class Fenix(Strategy):
        def init(self):
            super().init()
            #####   PIVOTS
            #self.pivot_high = self.I(indicador,self.data.pivot_high)
            #self.pivot_low = self.I(indicador,self.data.pivot_low)
            #self.decisional = self.I(indicador,self.data.decisional)
            #self.ema = self.I(ema,self.data.Close)
            #self.techo_del_minimo = self.I(indicador,self.data.techo_del_minimo)
            #self.piso_del_maximo = self.I(indicador,self.data.piso_del_maximo)
            #self.bos_bajista = self.I(indicador,self.data.bos_bajista)
            #self.bos_alcista = self.I(indicador,self.data.bos_alcista)
            #####   IMBALANCES
            self.imba_bajista_low = self.I(indicador,self.data.imba_bajista_low)
            self.imba_bajista_high = self.I(indicador,self.data.imba_bajista_high)
            self.imba_alcista_low = self.I(indicador,self.data.imba_alcista_low)
            self.imba_alcista_high = self.I(indicador,self.data.imba_alcista_high)
        def next(self):       
            super().next()
            if self.position:
                if self.data.cierra[-1]==True:
                    self.position.close()                    
            else:   
                if np.isnan(data.take_profit[-1]):
                    tp_value = None
                else:
                    tp_value = self.data.take_profit[-1]
                size= balance*self.data.porcentajeentrada[-1]/100
                if self.data.signal[-1]==1:
                    self.buy(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
                elif self.data.signal[-1]==-1:
                    self.sell(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
    bt = Backtest(data, Fenix, cash=balance)
    output = bt.run()
    if plot_flag:
        bt.plot()
    return output

####################################################################################

def smart_money(df):
    try:
        # Parámetros de longitud para los puntos pivote High y Low
        leftLenH    = 1
        rightLenH   = 1
        leftLenL    = 1
        rightLenL   = 1
        df['pivot_high'] = np.NaN
        df['pivot_low'] = np.NaN
        df['row_number'] = (range(len(df)))
        df.set_index('row_number', inplace=True)
        ## PIVOTS SUPERIORES
        for i in range(leftLenH, len(df) - rightLenH):
            if (
                df['High'].iloc[i] == df['High'].iloc[i - leftLenH:i + rightLenH + 1].max()
                and df['High'].iloc[i] > df['High'].iloc[i - 1]
                and df['High'].iloc[i] > df['High'].iloc[i + 1]
                ):
                df.at[i, 'pivot_high'] = df['High'].iloc[i]
        ## PIVOTS INFERIORES
        for i in range(leftLenL, len(df) - rightLenL):
            if (
                df['Low'].iloc[i] == df['Low'].iloc[i - leftLenL:i + rightLenL + 1].min()
                and df['Low'].iloc[i] < df['Low'].iloc[i - 1]
                and df['Low'].iloc[i] < df['Low'].iloc[i + 1]
                ):
                df.at[i, 'pivot_low'] = df['Low'].iloc[i]
        ## RELLENO DE PIVOTS
        for i in range(0, len(df)):
            if np.isnan(df['pivot_low'].iloc[i]):
                df.at[i, 'pivot_low'] = df['pivot_low'].iloc[i - 1]
            if np.isnan(df['pivot_high'].iloc[i]):
                df.at[i, 'pivot_high'] = df['pivot_high'].iloc[i - 1]   
        #####################
        # DECISIONALES
        df['color'] = np.where(df.Close > df.Open,'verde','rojo')
        df['decisional'] = np.where(                                
                                    (df.color == 'verde') # vela actual verde
                                    & (df.color.shift(1) == 'rojo') # vela anterior roja
                                    & (df.color.shift(-2) == 'rojo') # segunda vela roja                                
                                    & (df.High.shift(-3) < df.Low)    
                                    & (df.atr >= df.Close-df.Open)
                                    ,df.Low,
                                    np.where(
                                            (df.color == 'rojo') 
                                            & (df.color.shift(1) == 'verde') 
                                            & (df.color.shift(-2) == 'verde')
                                            & (df.Low.shift(-3) > df.High)                
                                            & (df.atr >= df.Open-df.Close)
                                            ,df.High,
                                            np.NaN)
                                    )
        df['decisional_flag'] = df['decisional']
        for i in range(0, len(df)):
            if np.isnan(df['decisional'].iloc[i]):
                df.at[i, 'decisional'] = df['decisional'].iloc[i - 1]
        ################# IMBALANCES
        df4h = md.obtiene_historial(symbol,'4h')
        df4h['row_number'] = (range(len(df4h)))
        df4h.set_index('row_number', inplace=True)
        df4h['imba_bajista_high'] = np.where(
                                        ((df4h.Low.shift(1)) >= (df4h.High.shift(-1)+df4h.atr))                  
                                        ,df4h.Low.shift(1),np.NaN)
        df4h['imba_bajista_low'] = np.where(np.isnan(df4h['imba_bajista_high']),np.NaN,df4h.High.shift(-1))
        df4h['imba_alcista_high'] = np.where(
                                        ((df4h.High.shift(1)) <= (df4h.Low.shift(-1)-df4h.atr))
                                        ,df4h.Low.shift(-1),np.NaN)
        df4h['imba_alcista_low'] = np.where(np.isnan(df4h['imba_alcista_high']),np.NaN,df4h.High.shift(1))
        df4h['timestamp']=df4h['Open Time']
        df4h.set_index('timestamp', inplace=True)    
        df4h=df4h[['imba_bajista_high','imba_bajista_low','imba_alcista_high','imba_alcista_low','Open Time']]
        df=pd.merge(df,df4h, on=["Open Time"], how='left')        
            ## RELLENO Y BORRADO DE IMBALANCES MITIGADOS
            # imbalance alcista
        imbalance_creado = False
        imba_alcista_high = np.nan
        imba_alcista_low = np.nan
        for i in range(0, len(df)):
            if not np.isnan(df['imba_alcista_high'].iloc[i]): 
                # un imbalance detectado en timeframe de 4h. Guardo los valores
                imba_alcista_high=df['imba_alcista_high'].iloc[i]
                imba_alcista_low=df['imba_alcista_low'].iloc[i]   
                imbalance_creado = False
            if imbalance_creado == False:     
                # no es un imbalance detectado asi que estiro el dibujo
                if df.Close.iloc[i] > imba_alcista_low: ## ya se creó el imbalance detectado. Dibujo
                    df.at[i, 'imba_alcista_high'] = imba_alcista_high
                    df.at[i, 'imba_alcista_low'] = imba_alcista_low
                    imbalance_creado = True
            else:
                if df.Close.iloc[i] > imba_alcista_low: ## ya se completó el imbalance detectado. Dibujo mientras no sea mitigado
                    df.at[i, 'imba_alcista_high'] = imba_alcista_high
                    df.at[i, 'imba_alcista_low'] = imba_alcista_low
                else:
                    # se mitigó el imbalance, no dibujo
                    imba_alcista_high = np.nan
                    imba_alcista_low = np.nan
            # imbalance bajista
        imbalance_creado = False
        imba_bajista_high = np.nan
        imba_bajista_low = np.nan
        for i in range(0, len(df)):
            if not np.isnan(df['imba_bajista_high'].iloc[i]): 
                # un imbalance detectado en timeframe de 4h. Guardo los valores
                imba_bajista_high=df['imba_bajista_high'].iloc[i]
                imba_bajista_low=df['imba_bajista_low'].iloc[i]   
                imbalance_creado = False
            if imbalance_creado == False:     
                # no es un imbalance detectado asi que estiro el dibujo
                if df.Close.iloc[i] < imba_bajista_high: ## ya se creó el imbalance detectado. Dibujo
                    df.at[i, 'imba_bajista_high'] = imba_bajista_high
                    df.at[i, 'imba_bajista_low'] = imba_bajista_low
                    imbalance_creado = True
            else:
                if df.Close.iloc[i] < imba_bajista_high: ## ya se completó el imbalance detectado. Dibujo mientras no sea mitigado
                    df.at[i, 'imba_bajista_high'] = imba_bajista_high
                    df.at[i, 'imba_bajista_low'] = imba_bajista_low
                else:
                    # se mitigó el imbalance, no dibujo
                    imba_bajista_high = np.nan
                    imba_bajista_low = np.nan            


        ################ VARIABLES PARA BOS
        ### BOS BAJISTA
        pico_maximo = 0
        piso_del_maximo = 0
        df['piso_del_maximo'] = np.NaN
        for i in range(0, len(df)):
            if df['pivot_high'].iloc[i]>pico_maximo:
                pico_maximo = df['pivot_high'].iloc[i]
                piso_del_maximo = df['pivot_low'].iloc[i]
                df.at[i, 'piso_del_maximo'] = piso_del_maximo
            if df['Close'].iloc[i] < piso_del_maximo:
                pico_maximo = 0
                piso_del_maximo = 0                
                ##relleno
        for i in range(0, len(df)):
            if np.isnan(df['piso_del_maximo'].iloc[i]):
                df.at[i, 'piso_del_maximo'] = df['piso_del_maximo'].iloc[i - 1]
        df['bos_bajista']=df.piso_del_maximo
        bos=0
        for i in range(len(df)-1, 0,-1):
            if (df['Close'].iloc[i] < df['piso_del_maximo'].iloc[i]):
                bos = df['piso_del_maximo'].iloc[i]
            if df['bos_bajista'].iloc[i]!=bos:
                df.at[i, 'bos_bajista'] = np.NaN
        ### BOS ALCISTA
        pico_minimo = float('inf')
        techo_del_minimo = float('inf')
        df['techo_del_minimo'] = np.NaN
        for i in range(0, len(df)):
            if df['pivot_low'].iloc[i]<pico_minimo:
                pico_minimo = df['pivot_low'].iloc[i]
                techo_del_minimo = df['pivot_high'].iloc[i]
                df.at[i, 'techo_del_minimo'] = techo_del_minimo
            if df['Close'].iloc[i] > techo_del_minimo:
                pico_minimo = float('inf')
                techo_del_minimo = float('inf')               
                ##relleno
        for i in range(0, len(df)):
            if np.isnan(df['techo_del_minimo'].iloc[i]):
                df.at[i, 'techo_del_minimo'] = df['techo_del_minimo'].iloc[i - 1]
        df['bos_alcista']=df.techo_del_minimo
        bos=0
        for i in range(len(df)-1, 0,-1):
            if (df['Close'].iloc[i] > df['techo_del_minimo'].iloc[i]):
                bos = df['techo_del_minimo'].iloc[i]
            if df['bos_alcista'].iloc[i]!=bos:
                df.at[i, 'bos_alcista'] = np.NaN                
        ## INDICE
        df['timestamp']=df['Open Time']
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass
##########################################################################################

def estrategia_royal(symbol,debug = False):
    try:                
        np.seterr(divide='ignore', invalid='ignore')
        # temporalidad de 1h para encontrar martillos y soportes/resistencias
        data = md.obtiene_historial(symbol,'15m')
        data=smart_money(data)          
        data.drop(['ema20','ema50', 'ema200'], axis=1, inplace=True)    
        data['signal'] = 0
        data['take_profit'] = np.NaN
        data['stop_loss'] = np.NaN
        data['cierra'] = False
        # Reemplazar valores no finitos (NA e inf) con 0
        data['porcentajeentrada'] = np.nan_to_num((data.Close/data.atr), nan=0, posinf=0, neginf=0)
        # Aplicar np.floor y convertir a enteros
        data['porcentajeentrada'] = np.floor(data['porcentajeentrada']).astype(int)
        ####################### alertas y valores
        if debug:
            df_str = data[list(data.columns)].to_string(index=False)
            print(df_str)
        return data
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass

#######################################################################################################################

symbol='BTCUSDT'
data = estrategia_royal(symbol)
backtesting(data, plot_flag=True)

#for index, values in data.iterrows():
#    if not np.isnan(values.decisional_flag):
#        print(f"{values['Open Time']} | {values.Close} | {values.pivot_low} | {values.decisional_flag}")

data[['Open Time', 
       'pivot_high', 'pivot_low', 
       'imba_bajista_high', 'imba_bajista_low', 'imba_alcista_high',
       'imba_alcista_low', 'piso_del_maximo', 'bos_bajista',
       'techo_del_minimo', 'bos_alcista']].tail(60)

C:\Users\Luis Gomez Freites\AppData\Local\Temp\ipykernel_10560\1276238474.py:50: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days

,Open Time,pivot_high,pivot_low,imba_bajista_high,imba_bajista_low,imba_alcista_high,imba_alcista_low,piso_del_maximo,bos_bajista,techo_del_minimo,bos_alcista
2023-09-21 04:15:00,2023-09-21 04:15:00,26985.9,26977.2,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 04:30:00,2023-09-21 04:30:00,26985.9,26977.2,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 04:45:00,2023-09-21 04:45:00,27092.4,26977.2,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 05:00:00,2023-09-21 05:00:00,27092.4,26977.2,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 05:15:00,2023-09-21 05:15:00,27092.4,27020.4,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 05:30:00,2023-09-21 05:30:00,27092.4,27020.4,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 05:45:00,2023-09-21 05:45:00,27077.0,27020.4,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 06:00:00,2023-09-21 06:00:00,27077.0,27020.4,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 06:15:00,2023-09-21 06:15:00,27077.0,27020.4,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN
2023-09-21 06:30:00,2023-09-21 06:30:00,27077.0,26990.0,NaN,NaN,NaN,NaN,26740.0,26740.0,27470.0,NaN


In [7]:
data.columns

Index(['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'atr',
       'pivot_high', 'pivot_low', 'color', 'decisional', 'decisional_flag',
       'imba_bajista_high', 'imba_bajista_low', 'imba_alcista_high',
       'imba_alcista_low', 'piso_del_maximo', 'bos_bajista',
       'techo_del_minimo', 'bos_alcista', 'signal', 'take_profit', 'stop_loss',
       'cierra', 'porcentajeentrada'],
      dtype='object')